<a href="https://colab.research.google.com/github/mohith-vishwanath/Image_Classification_Kaggle_Cats_vs_Dogs/blob/main/Tensorflow_Cats_vs_Dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import tqdm

import itertools
import os, shutil, time
import random, glob
import matplotlib.pyplot as plt
import cv2

from google.colab import drive
drive.mount('/content/drive')

Checking GPU availability

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices("GPU")
print("Number of GPUs available : ",len(physical_devices))
try:
    tf.config.experimental.set_memory_growth(physical_devices[0],True)
except:
    pass

In [ ]:
test_images = glob.glob("/content/drive/MyDrive/Datasets/Cats_vs_Dogs/test1/*.jpg")
train_images = glob.glob("/content/drive/MyDrive/Datasets/Cats_vs_Dogs/train/*.jpg")

In [ ]:
len(test_images),len(train_images)

In [ ]:
take_subset_images = True

if take_subset_images:
    all_images = train_images[:1000]        
else:
    all_images = train_images

In [ ]:
IMG_SIZE = 224
CATEGORIES = ["cat","dog"]
BATCH_SIZE = 100
VAL_SPLIT = 0.2
EPOCHS = 5

In [ ]:
manual_processing = False

if manual_processing:
    input_images = []
    labels = []
    for image in tqdm.tqdm(all_images,desc="Processing images "):

        name = str(image).split("/")[-1]
        name = str(name).split(".")[0]
        labels.append(CATEGORIES.index(name))

        img_array = cv2.imread(image,cv2.IMREAD_GRAYSCALE)
        img_array = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
        input_images.append(img_array)

else:

    if os.path.isdir("/content/drive/MyDrive/Datasets/Cats_vs_Dogs/training/dog") is False:
        os.makedirs("/content/drive/MyDrive/Datasets/Cats_vs_Dogs/training/dog")
        os.makedirs("/content/drive/MyDrive/Datasets/Cats_vs_Dogs/training/cat")

        for image in tqdm.tqdm(train_images,desc="Moving images "):
            name = str(image).split("/")[-1]
            name = str(name).split(".")[0]
            shutil.move(image,"/content/drive/MyDrive/Datasets/Cats_vs_Dogs/training/"+name+"/")

    train_path = "/content/drive/MyDrive/Datasets/Cats_vs_Dogs/training"

    datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input,validation_split=VAL_SPLIT)

    training_batches = datagen.flow_from_directory(directory=train_path,target_size=(IMG_SIZE,IMG_SIZE),classes=CATEGORIES,batch_size=BATCH_SIZE,subset="training")
    validation_batches = datagen.flow_from_directory(directory=train_path,target_size=(IMG_SIZE,IMG_SIZE),classes=CATEGORIES,batch_size=BATCH_SIZE,subset="validation")

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1,10,figsize=(20,20))
    axes = axes.flatten()
    for img,ax in zip(images_arr,axes):
        ax.imshow(img)
        ax.axis("off")
    plt.tight_layout()
    plt.show()

In [ ]:
img,target = next(training_batches)
plotImages(img)

Build Model

In [ ]:
model = Sequential([
    Conv2D(filters=64, kernel_size=(3,3),activation="relu",padding="same",input_shape=(IMG_SIZE,IMG_SIZE,3)),
    MaxPool2D(pool_size=(2,2),strides=2),
    Conv2D(filters=128,kernel_size=(3,3),activation="relu",padding="same"),
    MaxPool2D(pool_size=(2,2),strides=2),
    Flatten(),
    Dense(units=4096,activation="relu")
])

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001),loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
model.fit(x=training_batches,validation_data=validation_batches,epochs=10,verbose=1)